In [ ]:
import pretty_midi
import numpy as np
import os
import IPython
import fluidsynth
import glob
import pandas as pd

In [ ]:
song_list = glob.glob(os.path.join("data", "*.mid"))
song_list

In [ ]:
len(song_list)

In [ ]:
def bpm_to_120(midi_file):
    """
    This function evens out the tempo throuout the song to be 120 BPM,
    even if there are tempo changes. It gets a midi file as an input
    and outputs a modified pretty_midi object and the song title as a string.
    """

    mid = pretty_midi.PrettyMIDI(midi_file)

    tempo = mid.get_tempo_changes()
    num_of_changes = len(tempo[0])
    full_length = mid.get_end_time()

    old_times = []
    changes = [0]

    for i in range(num_of_changes):
        old_times.append(tempo[0][i])
        if i < (num_of_changes - 1):
            changes.append((tempo[0][i+1] - tempo[0][i]) * (tempo[1][i]/120))
        else:
            changes.append((full_length - tempo[0][i]) * (tempo[1][i]/120))

    old_times.append(full_length)
    new_times = np.cumsum(changes)

    mid.adjust_times(old_times, new_times)

    song_title = os.path.splitext(os.path.basename(midi_file))[0]

    return mid, song_title

In [ ]:
%time
guitar_tracks = []
drum_tracks = []
for song in song_list:
    # print(song)
    mid, song_title = bpm_to_120(song)
    song_dict = extract_guitar_and_drums(mid, song_title)
    guitar_tracks.append(song_dict['guitar'])
    drum_tracks.append(song_dict['drums'])


In [ ]:
len(guitar_tracks)

In [ ]:
midi_df = pd.DataFrame({'song_name': song_list, 'guitar_tracks': guitar_tracks, 'drum_tracks': drum_tracks})
midi_df

In [ ]:
mid_120, title = bpm_to_120('data/The_Fortunes_-_Youve_Got_Your_Troubles.mid')

In [ ]:
mid_120.instruments

In [ ]:
def extract_guitar_and_drums(mid, song_title='unknown'):
    """This function extracts the guitar and drum tracks from a midi file.
       The first input is either a path to a midi file (for example: 'raw_data/song_name.mid')
       in string format, or a pretty_midi object. The second input is the song title as a string.
       The output is a dictionary with the song name, guitar track and drum track"""

    if type(mid) == str:
        mid = pretty_midi.PrettyMIDI(mid)
        song_title = os.path.splitext(os.path.basename(mid))[0]

    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []

    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))

        if (instrument.program >= 24) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))

    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]

    # song_title = os.path.splitext(os.path.basename(midi_file))[0]




    song_dict = {'title': song_title,
                 'down_beats': mid.get_downbeats(),
                 'guitar': guitar_track,
                 'drums': drum_track
                }
    return song_dict

In [ ]:
original = pretty_midi.PrettyMIDI('raw_data/Another-One-Bites-The-Dust-1.mid')

In [ ]:
original.get_downbeats()

In [ ]:
song_dict = extract_guitar_and_drums(mid_120, title)
song_dict

In [ ]:
def tracks_to_bars(song_dict: dict) -> dict:
    """This function accepts a dictionary as an input with 4 keys: 'title', 'down_beats', 'guitar', 'drums'.
    The function takes the guitar and drums, both pretty_midi instrument objects, and cuts them up into a sequence of individual bars.
    The output is a dictionary that contains the following keys/values: song_title, a list of guitar bars, a list of drum bars, and a list of the song's downbeats
    """

    new_dict={}
    new_dict['song_title']=song_dict['title']
    guitar = song_dict['guitar']
    drums = song_dict['drums']
    down_beats_array = song_dict['down_beats']


    guitar_bars_list = []
    drums_bars_list = []

    for i in range(len(down_beats_array)):
        if i < len(down_beats_array) - 1:
            end_time = down_beats_array[i+1]
        else:
            end_time = down_beats_array[i] + 2

        guitar_bar = []
        drums_bar = []
        for j in range(len(guitar.notes)):
            if (guitar.notes[j].start >= down_beats_array[i]) and (guitar.notes[j].end < end_time):
                guitar_bar.append(guitar.notes[j])

        for k in range(len(drums.notes)):
            if (drums.notes[k].start >= down_beats_array[i]) and (drums.notes[k].end < end_time):
                drums_bar.append(drums.notes[k])

        drums_bars_list.append(drums_bar)
        guitar_bars_list.append(guitar_bar)


    new_dict['guitar_bars'] = guitar_bars_list
    new_dict['drum_bars'] = drums_bars_list
    new_dict['down_beats'] = down_beats_array.tolist()
    return new_dict

In [ ]:
%time
std_guitar_bars = []
std_drum_bars = []
for song in song_list:
    # print(song)
    mid, song_title = bpm_to_120(song)
    song_dict = extract_guitar_and_drums(mid, song_title)
    new_dict = tracks_to_bars(song_dict)

    std_guitar = standardize_bars(new_dict['guitar_bars'], new_dict['down_beats'])
    std_drums = standardize_bars(new_dict['drum_bars'], new_dict['down_beats'])

    std_guitar_bars.append(std_guitar)
    std_drum_bars.append(std_drums)

In [ ]:
len(std_guitar_bars)

In [ ]:
len(std_drum_bars)

In [ ]:
midi_df['standardize_guitar_bars'] = std_guitar_bars
midi_df

In [ ]:
midi_df['standardize_drum_bars'] = std_drum_bars
midi_df

In [ ]:
debug_track = midi_df['guitar_tracks'].iloc[2]
debug_track

In [ ]:
midi_df.to_csv('midi_df.csv', escapechar='\\')

In [ ]:
def standardize_bars(list_of_bars, downbeats):
    """
    This function standardizes the timing of musical bars
    so that each bar will start at the same time point.
    It gets a list of bars and the list of downbeats as inputs
    and returns a list of bars that all start with time = 1
    """

    for i in range(len(list_of_bars)):
        for j in range(len(list_of_bars[i])):
            if i == 0:
                list_of_bars[0][j].start = list_of_bars[0][j].start / downbeats[1]
                list_of_bars[0][j].end = list_of_bars[0][j].end / downbeats[1]

            list_of_bars[i][j].start = list_of_bars[i][j].start / downbeats[i]
            list_of_bars[i][j].end = list_of_bars[i][j].end / downbeats[i]

    return list_of_bars


In [ ]:
def mapping_dictionary(list_of_bars):
    bar_dict = {}
    for bar in list_of_bars:
        bar_dict[str(bar)] = bar

    return bar_dict

In [ ]:
bar_dict = mapping_dictionary(standardized_drums)
bar_dict

In [ ]:
len(bar_dict.keys())

In [ ]:
def objects_to_strings(list_of_bars):
    list_of_strings = [str(bar) for bar in list_of_bars]
    return list_of_strings

In [ ]:
len(strings)

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer
#from transformers import PreTrainedTokenizerFast
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
#tokenizer.pre_tokenizer = WhitespaceSplit()
training_data = strings
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

#training_corpus = get_training_corpus()
tokenizer.train_from_iterator(training_data, trainer=trainer)

In [ ]:
len(tokenizer.get_vocab())

In [ ]:
import time

In [ ]:
start = time.time()

string_guitars = []
string_drums = []
for i in range(len(std_guitar_bars)):
    list_of_strings_guitar = objects_to_strings(std_guitar_bars[i])
    list_of_strings_drums = objects_to_strings(std_guitar_bars[i])

    string_guitars.append(list_of_strings_guitar)
    string_drums.append(list_of_strings_drums)

end = time.time()
print(end - start)

In [ ]:
start = time.time()

string_guitars_all = []
string_drums_all = []
for i in range(len(std_guitar_bars)):
    string_guitars_all += string_guitars[i]
    string_drums_all += string_drums[i]

end = time.time()
print(end - start)

In [ ]:
len(set(string_guitars_all))

In [ ]:
guitar_tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
drum_tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))

guitar_trainer = WordLevelTrainer(vocab_size=len(set(string_guitars_all)),
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

drum_trainer = WordLevelTrainer(vocab_size=len(set(string_drums_all)),
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

guitar_tokenizer.train_from_iterator(string_guitars_all, trainer=guitar_trainer)
drum_tokenizer.train_from_iterator(string_drums_all, trainer=drum_trainer)


In [ ]:
guitar_tokenizer.save("guitar_tokenizer.json", pretty=True)
drum_tokenizer.save("drum_tokenizer.json", pretty=True)

In [ ]:
guitar_tokenizer.get_vocab()['[Note(start=1.291667, end=1.320660, pitch=64, velocity=125), Note(start=1.208333, end=1.327604, pitch=60, velocity=123)]']

In [ ]:
len(guitar_tokenizer.get_vocab())

In [ ]:
string_guitars[0]

In [ ]:
def guitar_encoder(guitar_track):
    tokenized_guitar = []
    for bar in guitar_track:
        tokenized_guitar.append(guitar_tokenizer.get_vocab()[bar])

    return tokenized_guitar

In [ ]:
def drum_encoder(drum_track):
    tokenized_drum = []
    for bar in drum_track:
        tokenized_drum.append(drum_tokenizer.get_vocab()[bar])

    return tokenized_drum

In [ ]:
start = time.time()

guitar_token_list = []
drum_token_list = []
for i in range(len(string_guitars)):
    guitar_token_list.append(guitar_encoder(string_guitars[i]))
    drum_token_list.append(drum_encoder(string_drums[i]))

end = time.time()
print(end - start)

In [ ]:
len(tokenized_guitar)

In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

# Initialize a tokenizer
guitar_tokenizer = Tokenizer(models.BPE())

# Customize pre-tokenization and decoding
# guitar_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
# guitar_tokenizer.decoder = decoders.ByteLevel()
# guitar_tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)

# And then train
trainer = trainers.BpeTrainer(
    vocab_size=20000,
    min_frequency=2
)
guitar_tokenizer.train_from_iterator(string_guitars_all, trainer=trainer)

# And Save it
#guitar_tokenizer.save("guitar_tokenizer.json", pretty=True)

In [ ]:
test_str = ''
for jj in range(len(string_guitars[0])):
    if jj == 0:
        test_str = test_str + string_guitars[0][jj]

    test_str = test_str + ' ' + string_guitars[0][jj]

test_str

In [ ]:
tt = guitar_tokenizer.encode(test_str)
tt

In [ ]:
tt.tokens

In [ ]:
len(string_guitars[0])